# **Crawling Data**

> Data crawling adalah proses automatis untuk mengumpulkan dan mengindeks data dari berbagai sumber seperti situs web, database, atau dokumen. Proses ini kemudian mengimpor informasi atau data yang telah ditemukan ke dalam file lokal di komputer Anda. Data yang dikumpulkan melalui crawling kemudian dapat diproses dan digunakan untuk berbagai tujuan, seperti analisis data, penelitian, atau pengembangan sistem informasi.

>  Data scraping atau pengikisan data adalah proses pengambilan data yang available, baik itu di web atau komputer. Proses ini kemudian dapat mengimpor informasi yang telah ditemukan ke dalam file lokal komputer Anda. Data-data yang telah terkumpul terkadang juga dapat disalurkan ke situs web lain. Scraping dilakukan saat data sudah terkumpul melalui proses crawling.

> Data scraping hanya mengambil data yang dipilih saat proses pengunduhan, sedangkan proses ini hanya memilah target yang akan dipilih. Proses crawling memungkinkan untuk meminimalisir reduplikasi data, berbeda dengan scraping yang masih rawan akan adanya reduplikasi karena selalu ada konten online yang diduplikasi di beberapa website.

> Libary untuk scraping tersedia banyak, namun untuk kasus ini menggunakan library "snscrape" dan "BeautifulSoup"

## Twitter

sncrape

In [ ]:
!pip install snscrape

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 KB 4.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import snscrape.modules.twitter as sracp

In [ ]:
topik = "prabowo"
tweets = []
limit = 500
for tweet in sracp.TwitterSearchScraper(topik).get_items():
  if limit==0:
    break
  else:
    tweets.append([tweet.user,tweet.content,tweet.place])
    limit-=1
df = pd.DataFrame(tweets, columns=["Akun","Tweet","Lokasi"])
df

<ipython-input-70-469bd80b5bfb>:8: DeprecatedFeatureWarning: content is deprecated, use rawContent instead
  tweets.append([tweet.user,tweet.content,tweet.place])


,Akun,Tweet,Lokasi
0,https://twitter.com/sastra_diaz,@bumnbersatu @mohmahfudmd @jokowi @Dennysirega...,None
1,https://twitter.com/HERMAWA87270689,@Farrel1510 @Android_AK_47 @prabowo @jokowi Ga...,None
2,https://twitter.com/goentz1,"Dalam unggahan tersebut, Prabowo bersama Najwa...",None
3,https://twitter.com/neinaadella07,"Jika Pilpres dilaksanakan saat ini,” kata Pene...",None
4,https://twitter.com/neinaadella07,Survei Elektabilitas Prabowo Subianto di Pilpr...,None
...,...,...,...
495,https://twitter.com/PolsekCangkuang,Kapolri Jenderal Polisi Drs. Listyo Sigit Prab...,None
496,https://twitter.com/UnameOne,dipimpin langsung oleh Kepala Staf TNI AL Laks...,None
497,https://twitter.com/Christprab,"@hrdbacot Diinterview onlen bapak2 dari bank, ...",None
498,https://twitter.com/C9Toko,Kapolri Jenderal Polisi Drs. Listyo Sigit Prab...,None


In [ ]:
df.to_csv("twitter_prabowo.csv",index=False)

## PTA (pta.trunojoyo.ac.id)

In [ ]:
!pip install bs4

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [ ]:
class pta:
  def __init__(self, url):
    self.url = url
  
  def page(self):
    res = requests.get(self.url)
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find('ol', class_='pagination').findChildren(recursive=False)
    return int(contents[-3].text)

  def artikel_page(self, page):
    res = requests.get(f'{self.url}/{page}')
    soup = BeautifulSoup(res.content, 'html.parser')
    contents = soup.find_all('li', attrs={'data-cat':'#luxury'})
    return contents

  def get_berita(self):
    pta_list = []
    page = self.page()
    for pg in range(1, page+1):
      artikels = self.artikel_page(pg)
      for artikel in artikels:
        respone = requests.get(artikel.find('a', class_='gray button')['href'])
        soup_data = BeautifulSoup(respone.content, 'html.parser')
        cont = soup_data.find('div', attrs={'id':'content_journal'})
        # ambil data
        nama = cont.findAll('div')[1].div.span.text.split(' : ')[1]
        judul = str.join("", (cont.find('a', class_='title').string).splitlines()).capitalize()
        abstrak = str.join("", (cont.find('p').string).splitlines()).capitalize()
        pta_list.append({
            'Penulis' : nama,
            'Judul': judul,
            'Abstrak': abstrak
        })
    self.pta = pta_list
  
  def result(self):
    return pd.DataFrame(self.pta)
  
  def save(self, namefile='pta'):
    df = pd.DataFrame(self.pta).to_csv(namefile+'.csv', index=False)
    print('Berhasil di save')

In [ ]:
scPta = pta('https://pta.trunojoyo.ac.id/c_search/byprod/10/')

In [ ]:
scPta.get_berita()

In [ ]:
scPta.result()

,Penulis,Judul,Abstrak
0,A.Ubaidillah S.Kom,Perancangan dan implementasi sistem database t...,Sistem informasi akademik (siakad) merupaka...
1,"M. Basith Ardianto,",Aplikasi kontrol dan monitoring jaringan kompu...,Berjalannya koneksi jaringan komputer dengan l...
2,"Akhmad Suyandi, S.Kom",Rancang bangun aplikasi proxy server untukenkr...,Web server adalah sebuah perangkat lunak serve...
3,Heri Supriyanto,Sistem pendukung keputusan optimasi penjadwala...,Penjadwalan kuliah di perguruan tinggi me...
4,Septian Rahman Hakim,Sistem augmented reality animasi benda bergera...,Seiring perkembangan teknologi yang ada diduni...
5,Adi Chandra Laksono,Gerak pekerja pada game real time strategy men...,Gerak pekerja ada pada game yang memiliki genr...
6,NURRACHMAT,Rancang bangun game perawatan sapi karapan men...,"Perkembangan game yang semakin pesat, memberik..."
7,Muhammad Choirur Rozi,Ekstraksi fitur berbasis two dimensional linea...,Sistem pengenalan wajah adalah suatu sistem un...
8,M Khoiril Anwar,Implementasi algoritma prim dan depth first ...,Teknologi mobile game beroperating system open...
9,MALIKUL HAMZAH,Perancangan sistem informasi badan kepegawaian...,Kantor badan kepegawaian kota bangkalan adalah...


In [ ]:
scPta.save()

Berhasil di save


### CNBC

In [ ]:
import requests as req
from bs4 import BeautifulSoup as bs
from datetime import datetime
import csv
import pandas as pd

hades = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36'}

In [ ]:
def scrape_cnbc(hal):
    global hades
    a = 1
    for page in range(1,hal):
        url_cnbc = f"https://www.cnbcindonesia.com/search?query=pemilu&kanal=&tipe=&date="
        ge = req.get(url_cnbc,hades).text
        sop = bs(ge,'lxml')
        ul = sop.find('ul',class_='list media_rows middle thumb terbaru gtm_indeks_feed')
        li = ul.find_all('li')
        for x in li:
            link_art = x.find('article').find('a')['href']        
            ge_ = req.get(link_art,hades).text
            sop_ = bs(ge_,'lxml')
            art = sop_.find('div',class_='lm_content mt10').find('article')
            headline = art.find('h1').text
            warp = art.find_all('div',class_='detail_text')
            for x in warp:
                x = x.find_all('p')
                y  = [y.text for y in x ]
                content = ''.join(y).replace('\n', '').replace('ADVERTISEMENT','').replace('SCROLL TO RESUME CONTENT','')
                print(f'done[{a}] > {headline[0:10]}')
                a += 1
                with open('beritaPemilu.csv','a')as file:
                    wr = csv.writer(file, delimiter=',')
                    wr.writerow([headline,content])

In [ ]:
scrape_cnbc(6)

done[1] > SBY Koment
done[2] > Inflasi Tu
done[3] > Wamenkumha
done[4] > Heboh Penu
done[5] > PN Jakpus 
done[6] > Mahfud MD:
done[7] > Kabulkan G
done[8] > Ini Tema R
done[9] > Menteri PA
done[10] > Sri Mulyan
done[11] > Airlangga 
done[12] > Tantangan 
done[13] > SBY Koment
done[14] > Inflasi Tu
done[15] > Wamenkumha
done[16] > Heboh Penu
done[17] > PN Jakpus 
done[18] > Mahfud MD:
done[19] > Kabulkan G
done[20] > Ini Tema R
done[21] > Menteri PA
done[22] > Sri Mulyan
done[23] > Airlangga 
done[24] > Tantangan 
done[25] > SBY Koment
done[26] > Inflasi Tu
done[27] > Wamenkumha
done[28] > Heboh Penu
done[29] > PN Jakpus 
done[30] > Mahfud MD:
done[31] > Kabulkan G
done[32] > Ini Tema R
done[33] > Menteri PA
done[34] > Sri Mulyan
done[35] > Airlangga 
done[36] > Tantangan 
done[37] > SBY Koment
done[38] > Inflasi Tu
done[39] > Wamenkumha
done[40] > Heboh Penu
done[41] > PN Jakpus 
done[42] > Mahfud MD:
done[43] > Kabulkan G
done[44] > Ini Tema R
done[45] > Menteri PA
done[46] > Sri Muly

In [ ]:
df = pd.read_csv('/content/beritaPemilu.csv')
df

,SBY Komentari Putusan PN Jakpus: Ada yang Aneh di Negeri Ini,"Jakarta, CNBC Indonesia - Presiden ke-6 Republik Indonesia Susilo Bambang Yudhoyono (SBY) turut berkomentar perihal putusan Pengadilan Negeri Jakarta Pusat yang memerintahkan Komisi Pemilihan Umum menunda tahapan Pemilihan Umum 2024. Komentar itu disampaikan SBY melalui akun Twitter resmi, Jumat (3/3/2023).""Menyimak putusan Pengadilan Negeri Jakarta Pusat kemarin (tentang pemilu), rasanya ada yang aneh di negeri ini,"" ujarnya.Menurut SBY, banyak pikiran dan hal yang keluar dari akal sehat.""Apa yang sesungguhnya terjadi? What is really going on? Semoga tidak terjadi sesuatu yang tidak kita inginkan di tahun pemilu ini,"" katanya.Lebih lanjut, Ketua Majelis Tinggi Partai Demokrat itu bilang bangsa ini tengah diuji. Banyak godaan.""Tapi, ingat rakyat kita. Jangan ada yang bermain api, terbakar nanti. Jangan ada yang menabur angin, kena badai nanti. Let's save our constitution and our beloved country,"" ujarnya.\r\r\r\r[Gambas:Twitter]"
0,"Inflasi Turki Turun Lagi, Erdogan Full Senyum ...","Jakarta, CNBC Indonesia - Tingkat inflasi Turk..."
1,Wamenkumham 'Ogah' Komentari Putusan PN Jakpus...,"Jakarta, CNBC Indonesia - Wakil Menteri Hukum ..."
2,Heboh Penundaan Pemilu Hingga Mahfud Sebut Sen...,"Jakarta, CNBC Indonesia - Putusan Pengadilan N..."
3,"PN Jakpus Putuskan Penundaan Pemilu, Begini Re...","Jakarta, CNBC Indonesia - Kantor Staf Kepresid..."
4,Mahfud MD: PN Jakarta Pusat Membuat Sensasi ya...,"Jakarta, CNBC Indonesia - Menteri Koordinator ..."
5,"Kabulkan Gugatan, PN Jakpus Perintahkan KPU Tu...","Jakarta, CNBC Indonesia - Kabar mengejutkan da..."
6,Ini Tema Rencana Kerja Pemerintah Jokowi di Ta...,"Jakarta, CNBC Indonesia - Presiden Joko Widodo..."
7,"Menteri PANRB Lapor Nasib Honorer ke Jokowi, I...","Jakarta, CNBC Indonesia - Menteri Pendayagunaa..."
8,"Sri Mulyani Tak Cemas Harga Batu Bara Merosot,...","Jakarta, CNBC Indonesia - Setelah meroket sepa..."
9,Airlangga Ungkap Peluang Ekonomi di Tahun Politik,"Jakarta, CNBC Indonesia - Menteri Koordinator ..."
